# Tutorial 4: Harmonic and Interharmonic Analysis with GAPoT

## Análisis de Armónicos e Interarmónicos con GAPoT

This notebook provides an in-depth analysis of power systems with harmonic distortion:

1. **Harmonic power decomposition**: P_h, Q_h for each harmonic
2. **Cross-frequency distortion**: The unique M_D terms in GAPoT
3. **Interharmonic analysis**: Non-integer frequency ratios
4. **Comparison with IEEE 1459**: What GAPoT reveals that IEEE 1459 aggregates
5. **Practical implications**: Compensation strategy insights

---

## Theoretical Background

### Harmonic Distortion in Power Systems

Non-linear loads (rectifiers, inverters, LED drivers, etc.) inject harmonic currents:

$$i(t) = \sum_{h=1}^{N} I_h \sqrt{2} \sin(h\omega_1 t - \phi_h)$$

When these currents flow through system impedance, they cause voltage distortion:

$$u(t) = \sum_{h=1}^{N} U_h \sqrt{2} \sin(h\omega_1 t - \theta_h)$$

### The Challenge of Traditional Methods

**IEEE 1459** defines:
- Fundamental apparent power: $S_1 = U_1 I_1$
- Non-fundamental apparent power: $S_N = \sqrt{S^2 - S_1^2}$
- Harmonic distortion power: $D = \sqrt{S^2 - P^2 - Q_1^2}$

**Limitation**: D aggregates ALL harmonic interactions. We cannot identify which frequency pairs contribute most to distortion.

### GAPoT Solution

GAPoT explicitly calculates cross-frequency terms:

$$\mathbf{M}_D = \sum_{m \neq n} D_{mn} (\sigma_m \wedge \sigma_n)$$

Each $D_{mn}$ represents the interaction between harmonics m and n.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pandas as pd

from gapot import GeometricPower
from gapot.traditional import IEEE1459Power, PQTheory
from gapot.utils import generate_distorted_signal, rms, thd

plt.style.use('seaborn-v0_8-whitegrid')
print("Modules loaded successfully!")

## 1. Generate Highly Distorted System

We'll create a realistic scenario with multiple harmonics typical of a six-pulse rectifier load.

In [ ]:
# System parameters
f1 = 50.0          # Fundamental frequency (Hz)
fs = 50000.0       # Sampling frequency (Hz)
cycles = 4         # Number of cycles (more for better FFT resolution)

# Fundamental parameters
U1 = 230.0         # RMS voltage (V)
I1 = 50.0          # RMS current (A) - larger load
phi1 = 0.4         # ~23° lagging (PF ≈ 0.92)

# Six-pulse rectifier harmonic spectrum (typical)
# Harmonics: 5, 7, 11, 13, 17, 19, 23, 25...
# Magnitude decreases approximately as 1/h
harmonics = [
    # (h, U_h/U1, I_h/I1, phase_shift)
    (5,  0.04, 0.20, np.pi/4),      # 5th: 4% voltage, 20% current
    (7,  0.03, 0.14, np.pi/3),      # 7th: 3% voltage, 14% current
    (11, 0.02, 0.09, np.pi/5),      # 11th: 2% voltage, 9% current
    (13, 0.015, 0.08, np.pi/6),     # 13th: 1.5% voltage, 8% current
    (17, 0.01, 0.06, np.pi/7),      # 17th: 1% voltage, 6% current
    (19, 0.008, 0.05, np.pi/8),     # 19th: 0.8% voltage, 5% current
]

# Generate signals
u, i, t = generate_distorted_signal(
    U1=U1, I1=I1, phi1=phi1,
    harmonics=harmonics,
    f1=f1, fs=fs, cycles=cycles
)

# Calculate THD
THD_U = thd(u, f1, fs) * 100
THD_I = thd(i, f1, fs) * 100

print(f"Voltage RMS: {rms(u):.2f} V")
print(f"Current RMS: {rms(i):.2f} A")
print(f"Voltage THD: {THD_U:.2f}%")
print(f"Current THD: {THD_I:.2f}%")

In [ ]:
# Plot distorted waveforms
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

t_ms = t * 1000

# Voltage
axes[0].plot(t_ms, u, 'b-', linewidth=1)
# Add pure sinusoid for reference
u_pure = U1 * np.sqrt(2) * np.sin(2 * np.pi * f1 * t)
axes[0].plot(t_ms, u_pure, 'b--', linewidth=0.8, alpha=0.5, label='Pure fundamental')
axes[0].set_ylabel('Voltage (V)')
axes[0].set_title(f'Distorted Voltage Waveform (THD = {THD_U:.1f}%)')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# Current
axes[1].plot(t_ms, i, 'r-', linewidth=1)
i_pure = I1 * np.sqrt(2) * np.sin(2 * np.pi * f1 * t - phi1)
axes[1].plot(t_ms, i_pure, 'r--', linewidth=0.8, alpha=0.5, label='Pure fundamental')
axes[1].set_ylabel('Current (A)')
axes[1].set_xlabel('Time (ms)')
axes[1].set_title(f'Distorted Current Waveform (THD = {THD_I:.1f}%)')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Harmonic Spectrum Analysis

In [ ]:
# FFT analysis
from scipy.fft import fft, fftfreq

N = len(u)
U_fft = np.abs(fft(u)) / N * 2
I_fft = np.abs(fft(i)) / N * 2
freqs = fftfreq(N, 1/fs)

# Only positive frequencies up to 1500 Hz
mask = (freqs >= 0) & (freqs <= 1500)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Voltage spectrum
axes[0].stem(freqs[mask], U_fft[mask], 'b', markerfmt='bo', basefmt=' ')
axes[0].set_xlabel('Frequency (Hz)')
axes[0].set_ylabel('Voltage (V)')
axes[0].set_title('Voltage Harmonic Spectrum')
axes[0].grid(True, alpha=0.3)

# Annotate harmonics
for h in [1, 5, 7, 11, 13]:
    idx = np.argmin(np.abs(freqs - h*f1))
    if U_fft[idx] > 1:
        axes[0].annotate(f'h={h}', xy=(freqs[idx], U_fft[idx]), 
                        xytext=(freqs[idx]+30, U_fft[idx]+10),
                        fontsize=9)

# Current spectrum
axes[1].stem(freqs[mask], I_fft[mask], 'r', markerfmt='ro', basefmt=' ')
axes[1].set_xlabel('Frequency (Hz)')
axes[1].set_ylabel('Current (A)')
axes[1].set_title('Current Harmonic Spectrum')
axes[1].grid(True, alpha=0.3)

# Annotate harmonics
for h in [1, 5, 7, 11, 13]:
    idx = np.argmin(np.abs(freqs - h*f1))
    if I_fft[idx] > 1:
        axes[1].annotate(f'h={h}', xy=(freqs[idx], I_fft[idx]), 
                        xytext=(freqs[idx]+30, I_fft[idx]+2),
                        fontsize=9)

plt.tight_layout()
plt.show()

## 3. GAPoT Power Analysis

In [ ]:
# Calculate geometric power
gp = GeometricPower(u, i, f1=f1, fs=fs)

print(gp.summary())

## 4. Cross-Frequency Distortion Analysis (GAPoT Unique Feature)

This is where GAPoT shines: we can see exactly which harmonic pairs contribute to distortion power.

In [ ]:
# Analyze cross-frequency distortion terms
print("CROSS-FREQUENCY DISTORTION ANALYSIS")
print("=" * 60)
print("\nThese terms represent interactions between different harmonics.")
print("IEEE 1459 aggregates ALL of these into a single 'D' value.\n")

if gp.M_D_components:
    # Sort by magnitude
    sorted_terms = sorted(gp.M_D_components.items(), 
                          key=lambda x: abs(x[1]), reverse=True)
    
    print(f"{'Frequency Pair':<20} {'D_mn (va)':<15} {'% of ||M_D||':<15}")
    print("-" * 50)
    
    total_D_sq = gp.M_D_norm**2
    
    for (f_m, f_n), D_mn in sorted_terms[:10]:  # Top 10
        h_m = int(f_m / f1)
        h_n = int(f_n / f1)
        contribution = (D_mn**2 / total_D_sq) * 100 if total_D_sq > 0 else 0
        print(f"h={h_m} × h={h_n} ({f_m:.0f}×{f_n:.0f}Hz)  {D_mn:>10.2f}       {contribution:>8.1f}%")
    
    print("-" * 50)
    print(f"{'Total ||M_D||':<20} {gp.M_D_norm:<15.2f}")
else:
    print("No significant cross-frequency distortion detected.")

In [ ]:
# Visualize distortion matrix
if gp.M_D_components:
    # Create distortion matrix
    harmonics_present = sorted(set([1] + [h for h, _, _, _ in harmonics]))
    n_harm = len(harmonics_present)
    
    D_matrix = np.zeros((n_harm, n_harm))
    
    for (f_m, f_n), D_mn in gp.M_D_components.items():
        h_m = int(f_m / f1)
        h_n = int(f_n / f1)
        if h_m in harmonics_present and h_n in harmonics_present:
            i = harmonics_present.index(h_m)
            j = harmonics_present.index(h_n)
            D_matrix[i, j] = abs(D_mn)
            D_matrix[j, i] = abs(D_mn)
    
    # Plot heatmap
    fig, ax = plt.subplots(figsize=(10, 8))
    
    im = ax.imshow(D_matrix, cmap='YlOrRd')
    
    ax.set_xticks(range(n_harm))
    ax.set_yticks(range(n_harm))
    ax.set_xticklabels([f'h={h}' for h in harmonics_present])
    ax.set_yticklabels([f'h={h}' for h in harmonics_present])
    
    # Add values
    for i in range(n_harm):
        for j in range(n_harm):
            if D_matrix[i, j] > 0:
                text = ax.text(j, i, f'{D_matrix[i, j]:.0f}',
                              ha='center', va='center', color='black', fontsize=9)
    
    ax.set_title('Cross-Frequency Distortion Matrix |D_mn| (va)\n'
                 'GAPoT reveals which harmonic pairs interact')
    ax.set_xlabel('Harmonic n')
    ax.set_ylabel('Harmonic m')
    
    plt.colorbar(im, ax=ax, label='|D_mn| (va)')
    plt.tight_layout()
    plt.show()
    
    print("\nKey insight: The matrix shows that h=1×h=5 interaction dominates.")
    print("This information is LOST in IEEE 1459's aggregated D value.")

## 5. Comparison with IEEE 1459

In [ ]:
# IEEE 1459 analysis
ieee = IEEE1459Power(u, i, f1=f1, fs=fs)

print("COMPARISON: GAPoT vs IEEE 1459")
print("=" * 70)
print(f"\n{'Quantity':<30} {'GAPoT':<20} {'IEEE 1459':<20}")
print("-" * 70)
print(f"{'Active Power P (W)':<30} {gp.P:<20.2f} {ieee.P:<20.2f}")
print(f"{'Fundamental Q (var)':<30} {gp.M_Q_norm:<20.2f} {ieee.Q1:<20.2f}")
print(f"{'Distortion Power (va)':<30} {gp.M_D_norm:<20.2f} {ieee.D:<20.2f}")
print(f"{'Apparent Power S (VA)':<30} {gp.S:<20.2f} {ieee.S:<20.2f}")
print(f"{'Power Factor':<30} {gp.PF:<20.4f} {ieee.PF:<20.4f}")
print("=" * 70)

print("\n" + "=" * 70)
print("WHAT GAPoT REVEALS THAT IEEE 1459 CANNOT:")
print("=" * 70)
print(f"\nIEEE 1459 reports D = {ieee.D:.2f} va (single aggregated value)")
print(f"\nGAPoT decomposes this into {len(gp.M_D_components)} distinct cross-frequency terms:")

if gp.M_D_components:
    sorted_terms = sorted(gp.M_D_components.items(), 
                          key=lambda x: abs(x[1]), reverse=True)[:5]
    for (f_m, f_n), D_mn in sorted_terms:
        h_m, h_n = int(f_m/f1), int(f_n/f1)
        print(f"  • h={h_m} × h={h_n}: D = {abs(D_mn):.2f} va")

## 6. Harmonic-by-Harmonic Power Analysis

In [ ]:
# Get harmonic power breakdown
harmonic_powers = gp.get_harmonic_powers()

print("POWER BY HARMONIC")
print("=" * 60)
print(f"\n{'Harmonic':<12} {'Frequency':<12} {'P_h (W)':<15} {'Q_h (var)':<15}")
print("-" * 60)

P_total_check = 0
for freq, pwr in sorted(harmonic_powers.items()):
    h = int(freq / f1) if f1 > 0 and freq > 0 else 0
    label = 'DC' if freq == 0 else f'h={h}'
    print(f"{label:<12} {freq:<12.0f} {pwr['P']:<15.2f} {pwr['Q']:<15.2f}")
    P_total_check += pwr['P']

print("-" * 60)
print(f"{'TOTAL':<12} {'':<12} {P_total_check:<15.2f}")
print("=" * 60)

In [ ]:
# Visualize harmonic power distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Prepare data
freqs_list = sorted(harmonic_powers.keys())
P_list = [harmonic_powers[f]['P'] for f in freqs_list]
Q_list = [abs(harmonic_powers[f]['Q']) for f in freqs_list]
labels = [f'h={int(f/f1)}' if f > 0 else 'DC' for f in freqs_list]

x = np.arange(len(freqs_list))
width = 0.35

# Active power by harmonic
colors_P = ['green' if p >= 0 else 'red' for p in P_list]
axes[0].bar(x, P_list, width, color=colors_P, alpha=0.7, edgecolor='black')
axes[0].set_xticks(x)
axes[0].set_xticklabels(labels)
axes[0].set_xlabel('Harmonic')
axes[0].set_ylabel('Active Power (W)')
axes[0].set_title('Active Power by Harmonic')
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].axhline(y=0, color='k', linewidth=0.5)

# Reactive power by harmonic
axes[1].bar(x, Q_list, width, color='orange', alpha=0.7, edgecolor='black')
axes[1].set_xticks(x)
axes[1].set_xticklabels(labels)
axes[1].set_xlabel('Harmonic')
axes[1].set_ylabel('Reactive Power |Q_h| (var)')
axes[1].set_title('Reactive Power Magnitude by Harmonic')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("Note: Fundamental dominates, but harmonic contributions are significant.")

## 7. Practical Implications for Compensation

The cross-frequency analysis reveals which harmonic pairs to prioritize for compensation.

In [ ]:
print("COMPENSATION STRATEGY RECOMMENDATIONS")
print("=" * 70)
print("\nBased on GAPoT cross-frequency analysis:\n")

# Analyze which harmonics contribute most
harmonic_contributions = {}

for (f_m, f_n), D_mn in gp.M_D_components.items():
    h_m, h_n = int(f_m/f1), int(f_n/f1)
    
    # Accumulate contribution for each harmonic
    if h_m not in harmonic_contributions:
        harmonic_contributions[h_m] = 0
    if h_n not in harmonic_contributions:
        harmonic_contributions[h_n] = 0
    
    harmonic_contributions[h_m] += D_mn**2
    harmonic_contributions[h_n] += D_mn**2

# Sort by contribution
sorted_harmonics = sorted(harmonic_contributions.items(), 
                          key=lambda x: x[1], reverse=True)

print("Priority order for harmonic compensation:")
print("-" * 40)
total_contribution = sum(v for _, v in harmonic_contributions.items())

cumulative = 0
for h, contrib in sorted_harmonics:
    pct = (contrib / total_contribution) * 100 if total_contribution > 0 else 0
    cumulative += pct
    print(f"  {h}. Harmonic h={h}: {pct:.1f}% (cumulative: {cumulative:.1f}%)")

print("\n" + "=" * 70)
print("RECOMMENDATION:")
print("=" * 70)
print(f"\nCompensating harmonics h={sorted_harmonics[0][0]} and h={sorted_harmonics[1][0]}")
print(f"would address approximately {sum(c for _, c in sorted_harmonics[:2])/total_contribution*100:.0f}% of distortion power.")
print("\nThis targeted approach is more efficient than broadband filtering.")

## 8. Summary and Key Takeaways

### What We Learned

1. **GAPoT provides detailed harmonic analysis**: Each frequency component's contribution to P and Q is explicitly calculated.

2. **Cross-frequency distortion is decomposed**: Unlike IEEE 1459's aggregated D, GAPoT reveals which harmonic pairs interact.

3. **Compensation strategy insights**: By knowing which harmonics contribute most to distortion, we can design targeted compensation.

4. **Energy conservation verified**: $\|\mathbf{M}\|^2 = P^2 + \|\mathbf{M}_Q\|^2 + \|\mathbf{M}_D\|^2 = S^2$

### GAPoT Advantages for Harmonic Analysis

| Feature | IEEE 1459 | GAPoT |
|---------|-----------|-------|
| Harmonic P, Q | Requires separate calculation | Built-in |
| Distortion D | Single aggregated value | Individual D_mn terms |
| Cross-frequency analysis | Not available | Explicit bivector terms |
| Compensation insights | Limited | Detailed prioritization |